In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd

from file import ProcessGoEmotions, TokenizeDataset

/Users/cha/opt/anaconda3/envs/anlp/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PATH1 = "./data/full_dataset/goemotions_1.csv"
PATH2 = "./data/full_dataset/goemotions_2.csv"
PATH3 = "./data/full_dataset/goemotions_3.csv"

# LABEL = "emotions" # all 28 labels
LABEL = "emotion_category" # positive negative ambiguous and neutral

process = ProcessGoEmotions(label_choice=LABEL)
train_dataset, test_dataset = process.get_datasets(paths=[PATH1], test_size = 0.2)

In [3]:
tonekizer = TokenizeDataset(train_dataset)
tokenized_train = tonekizer.tokenize_process(tokenizer_name="bert-base-cased")

tonekizer = TokenizeDataset(test_dataset)
tokenized_test = tonekizer.tokenize_process(tokenizer_name="bert-base-cased")

In [4]:
small_train_dataset = tokenized_train.shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_test.shuffle(seed=42).select(range(100))

In [5]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

In [6]:
from model import Model

num_epochs = 3

model = Model(
    optimizer_name="AdamW",
    num_epochs=num_epochs,
    train_dataloader=train_dataloader,
    eval_dataloader=eval_dataloader,
    num_labels=len(tokenized_train['labels'].unique()),
    model_name="bert-base-cased"
)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [8]:
model.train()

100%|██████████| 39/39 [04:06<00:00,  6.32s/it]


In [10]:
result = model.evaluate(metric="accuracy")

In [11]:
result

{'accuracy': 0.35}